caffemodel 다운로드 

URL : https://github.com/habom2310/People-tracking-with-Age-and-Gender-detection

In [1]:
import glob
import dlib
import cv2

age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']  # 나이 분류 구간 정의
gender_list = ['Male','Female']  # 성별 구분 정의

# 얼굴을 감지하는 detector
detector = dlib.get_frontal_face_detector()      

# 나이를 감지하는 detector
age_detector = cv2.dnn.readNetFromCaffe(          
               'Data/caffe/deploy_age.prototxt',
               'Data/caffe/age_net.caffemodel')

# 성별을 감지하는 detector        
gender_detector = cv2.dnn.readNetFromCaffe(      
               'Data/caffe/deploy_gender.prototxt',
               'Data/caffe/gender_net.caffemodel')

img_list = glob.glob('Data/caffe/age_gender/*.jpg')  
len(img_list)



ModuleNotFoundError: No module named 'dlib'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for img_path in img_list:
  image = cv2.imread(img_path)  
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = detector(gray, 1)  # detector 에 의해 얼굴 감지
  print("Number of faces detected: {}".format(len(faces)))
  for face in faces:
    # boxing 좌표
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()  
    # 이미지에서 얼굴 영역만 copy
    face_img = image[y1:y2, x1:x2].copy()  
    blob = cv2.dnn.blobFromImage(face_img, scalefactor=1, size=(227, 227),
           mean=(78.4263377603, 87.7689143744, 114.895847746), swapRB=False, crop=False)
    # predict age
    age_detector.setInput(blob)
    age_preds = age_detector.forward()
    age = age_list[age_preds[0].argmax()]
    # predict gender
    gender_detector.setInput(blob)
    gender_preds = gender_detector.forward()
    gender = gender_list[gender_preds[0].argmax()]
    cv2.rectangle(image, (x1, y1), (x2, y2), (255,255,255), 2)
    overlay_text = '%s %s' % (gender, age)
    cv2.putText(image, overlay_text, org=(x1, y1), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1, color=(0,0,0), thickness=10)
    cv2.putText(image, overlay_text, org=(x1, y1),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255), thickness=2)
  image=cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)
  plt.figure()
  plt.imshow(image)
  plt.show()  

